In this notebook, experiments are run on the randomisation of hyperparameters


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils import data
import random
import copy
import numpy as np

In [ ]:
# GPU Stuff

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device = {device}")

Device = cuda


In [ ]:
class RandomOptimiser(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, num_layers):
        super(RandomOptimiser, self).__init__()
        self.num_layers = num_layers
        if num_layers == 3:
          self.fc1 = nn.Linear(input_size, hidden_size1)
          self.fc2 = nn.Linear(hidden_size1, hidden_size2)
          self.fc3 = nn.Linear(hidden_size2, hidden_size3)
          self.fc4 = nn.Linear(hidden_size3, 3)
        elif num_layers == 2:
          self.fc1 = nn.Linear(input_size, hidden_size1)
          self.fc2 = nn.Linear(hidden_size1, hidden_size2)
          self.fc3 = nn.Linear(hidden_size2, 3)
        else:
          self.fc1 = nn.Linear(input_size, hidden_size1)
          self.fc2 = nn.Linear(hidden_size1, 3)

    def forward(self, x):
        if self.num_layers == 3:
          x = torch.relu(self.fc1(x))
          x = torch.relu(self.fc2(x))
          x = torch.relu(self.fc3(x))
          x = self.fc4(x)
        elif self.num_layers == 2:
          x = torch.relu(self.fc1(x))
          x = torch.relu(self.fc2(x))
          x = self.fc3(x)
        else:
          x = torch.relu(self.fc1(x))
          x = self.fc2(x)
        return x

In [ ]:

def train_model(model, train_loader, valid_features, valid_labels, criterion, optimizer, epochs):
    train_losses = []
    train_accuracies = []
    valid_losses = []
    valid_accuracies = []

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0

        for batch_features, batch_labels in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_features)
            loss = criterion(outputs, batch_labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Calculate training metrics
        model.eval()
        with torch.no_grad():
            train_outputs = model(train_loader.dataset.tensors[0])
            train_loss = criterion(train_outputs, train_loader.dataset.tensors[1])
            train_predicted = torch.argmax(train_outputs, dim=1)
            train_accuracy = (train_predicted == train_loader.dataset.tensors[1]).float().mean()
            valid_outputs = model(valid_features)
            valid_loss = criterion(valid_outputs, valid_labels)
            valid_predicted = torch.argmax(valid_outputs, dim=1)
            valid_accuracy = (valid_predicted == valid_labels).float().mean()

        train_losses.append(running_loss / len(train_loader))
        train_accuracies.append(train_accuracy.item())
        valid_losses.append(valid_loss.item())
        valid_accuracies.append(valid_accuracy.item())

        if (epoch + 1) % 50 == 0:
            print(f'Epoch [{epoch + 1}/{epochs}], Train Loss: {train_losses[-1]:.4f}, Valid Loss: {valid_losses[-1]:.4f}')
            print(f'Train Accuracy: {train_accuracies[-1]:.4f}, Valid Accuracy: {valid_accuracies[-1]:.4f}')

    return train_losses, train_accuracies, valid_losses, valid_accuracies

In [ ]:
def random_search(train_features, train_labels, valid_features, valid_labels, n_trials=10):
    param_space = {
        'learning_rate': [0.0001, 0.001, 0.01, 0.1],
        'batch_size': [16, 32, 64],
        'epochs': [100, 200, 300],
        'hidden_size1': [16, 32, 64, 128],
        'hidden_size2': [16, 32, 64, 128],
        'hidden_size3': [16, 32, 64, 128],
        'no_hidden_layers': [1, 2, 3],
    }

    best_valid_accuracy = 0
    lowest_valid_loss = float('inf')
    best_model_accuracy = None
    best_model_loss = None
    best_params_accuracy = None
    best_params_loss = None
    best_no_hl = None

    input_size = train_features.shape[1]

    for trial in range(n_trials):
        current_params = {
            'learning_rate': random.choice(param_space['learning_rate']),
            'batch_size': random.choice(param_space['batch_size']),
            'epochs': random.choice(param_space['epochs']),
            'hidden_size1': random.choice(param_space['hidden_size1']),
            'hidden_size2': random.choice(param_space['hidden_size2']),
            'hidden_size3': random.choice(param_space['hidden_size3']),
            'no_hidden_layers': random.choice(param_space['no_hidden_layers']),
        }

        print(f"\nTrial {trial + 1}/{n_trials}")
        print("Current parameters:", current_params)

        accuracies = []
        losses = []
        for i in range(3):
          print(f"Sub-trial {i+1}:")
          model = RandomOptimiser(input_size, current_params['hidden_size1'], current_params['hidden_size2'], current_params['hidden_size3'],current_params['no_hidden_layers']).to(device)
          train_dataset = data.TensorDataset(train_features, train_labels)
          train_loader = data.DataLoader(train_dataset, batch_size=current_params['batch_size'], shuffle=True)

          # Training setup
          criterion = nn.CrossEntropyLoss()
          optimizer = optim.Adam(model.parameters(), lr=current_params['learning_rate'])

          # Train the model
          train_losses, train_accuracies, valid_losses, valid_accuracies = train_model(
              model, train_loader, valid_features, valid_labels, criterion, optimizer, current_params['epochs']
          )

          final_valid_accuracy = valid_accuracies[-1]
          final_valid_loss = valid_losses[-1]
          accuracies.append(final_valid_accuracy)
          losses.append(final_valid_loss)

        print(f"Accuracies: {accuracies}")
        print(f"Losses: {losses}")
        final_valid_accuracy = np.mean(accuracies)
        final_valid_loss = np.mean(losses)
        print(f"Average Accuracy: {final_valid_accuracy}")
        print(f"Average Loss: {final_valid_loss}")
        # Track the best validation accuracy
        if final_valid_accuracy > best_valid_accuracy:
            best_valid_accuracy = final_valid_accuracy
            best_model_accuracy = copy.deepcopy(model)
            best_params_accuracy = current_params
            print(f"New best model by Validation Accuracy! Accuracy: {best_valid_accuracy:.4f}")

        # Track the lowest validation loss
        if final_valid_loss < lowest_valid_loss:
            lowest_valid_loss = final_valid_loss
            best_model_loss = copy.deepcopy(model)
            best_params_loss = current_params
            print(f"New best model by Validation Loss! Loss: {lowest_valid_loss:.4f}")

    return best_model_accuracy, best_params_accuracy, best_valid_accuracy, best_model_loss, best_params_loss, lowest_valid_loss


In [ ]:

def process_data(data):
    features = []
    labels = []
    for line in data:
        if line.strip():
            split_line = list(map(float, line.split(',')))
            features.append(split_line[:-1])
            labels.append(int(split_line[-1]))
    return torch.tensor(features, dtype=torch.float32, device=device), torch.tensor(labels, dtype=torch.long, device=device)


In [ ]:

if __name__ == "__main__":

    TestingData = open('TestingEncoded.csv', 'r').read().split("\n")[1:]
    TrainingData = open('TrainingEncoded.csv', 'r').read().split("\n")[1:]
    ValidationData = open('ValidationEncoded.csv', 'r').read().split("\n")[1:]

    # Process data
    train_features, train_labels = process_data(TrainingData)
    valid_features, valid_labels = process_data(ValidationData)
    test_features, test_labels = process_data(TestingData)

    # Number of runs
    num_runs = 1

    best_model_accuracy_runs = None
    best_model_loss_runs = None
    best_params_accuracy_runs = None
    best_params_loss_runs = None
    best_valid_accuracy_runs = 0
    best_valid_loss_runs = float('inf')

    for run in range(num_runs):
        print(f"\nRun {run + 1}/{num_runs}")

        # Perform random search
        best_model_accuracy, best_params_accuracy, best_valid_accuracy, best_model_loss, best_params_loss, lowest_valid_loss = random_search(
            train_features, train_labels, valid_features, valid_labels, n_trials=30
        )

        # Compare across runs for best accuracy
        if best_valid_accuracy > best_valid_accuracy_runs:
            best_valid_accuracy_runs = best_valid_accuracy
            best_model_accuracy_runs = best_model_accuracy
            best_params_accuracy_runs = best_params_accuracy

        # Compare across runs for lowest validation loss
        if lowest_valid_loss < best_valid_loss_runs:
            best_valid_loss_runs = lowest_valid_loss
            best_model_loss_runs = best_model_loss
            best_params_loss_runs = best_params_loss

    # Print the best hyperparameters and results
    print("\nBest Validation Accuracy Across Runs:")
    print(f"Validation Accuracy: {best_valid_accuracy_runs:.4f}")
    print(f"Best Hyperparameters (Accuracy): {best_params_accuracy_runs}")

    print("\nLowest Validation Loss Across Runs:")
    print(f"Validation Loss: {best_valid_loss_runs:.4f}")
    print(f"Best Hyperparameters (Loss): {best_params_loss_runs}")

    # Save the models
    torch.save(best_model_accuracy_runs.state_dict(), 'best_model_accuracy.pth')
    torch.save(best_model_loss_runs.state_dict(), 'best_model_loss.pth')


Run 1/1

Trial 1/30
Current parameters: {'learning_rate': 0.1, 'batch_size': 16, 'epochs': 200, 'hidden_size1': 128, 'hidden_size2': 64, 'hidden_size3': 128, 'no_hidden_layers': 2}
Sub-trial 1:
Epoch [50/200], Train Loss: 1.0586, Valid Loss: 1.0630
Train Accuracy: 0.4837, Valid Accuracy: 0.4866
Epoch [100/200], Train Loss: 1.0592, Valid Loss: 1.0551
Train Accuracy: 0.4837, Valid Accuracy: 0.4866
Epoch [150/200], Train Loss: 1.0606, Valid Loss: 1.0528
Train Accuracy: 0.4837, Valid Accuracy: 0.4866
Epoch [200/200], Train Loss: 1.0589, Valid Loss: 1.0570
Train Accuracy: 0.4837, Valid Accuracy: 0.4866
Sub-trial 2:
Epoch [50/200], Train Loss: 1.0598, Valid Loss: 1.0618
Train Accuracy: 0.4837, Valid Accuracy: 0.4866
Epoch [100/200], Train Loss: 1.0586, Valid Loss: 1.0497
Train Accuracy: 0.4837, Valid Accuracy: 0.4866
Epoch [150/200], Train Loss: 1.0581, Valid Loss: 1.0503
Train Accuracy: 0.4837, Valid Accuracy: 0.4866
Epoch [200/200], Train Loss: 1.0589, Valid Loss: 1.0503
Train Accuracy: 0